In [1]:
# ── Full Reload, Retrain & Back-test (with Morning/Afternoon window) ────────────────────────
import importlib, pandas as pd
import algo.features as feat_mod
import algo.model    as model_mod
importlib.reload(feat_mod); importlib.reload(model_mod)

from algo.features   import add_indicators, FEATURES
from algo.model      import load_or_train, predict_last, LOOKBACK
from algo.broker     import KiteWrapper
from algo.config     import load_config
from algo.backtester import backtest

# 1️⃣ Pull 200 days of 3-min history and build features
cfg      = load_config()
broker   = KiteWrapper(cfg)
hist_all = broker.history(days=200, interval="3minute",
                          tradingsymbol="IDEA")
df_all   = add_indicators(hist_all, debug=True).ffill()

# 2️⃣ Split by *calendar days* instead of raw row count
n_test_days = 4
cutoff      = df_all.index[-1] - pd.Timedelta(days=n_test_days)
df_tr       = df_all[df_all.index <  cutoff]
df_te       = df_all[df_all.index >= cutoff]

print(f"Training rows: {len(df_tr):,}")
print(f"Testing  rows: {len(df_te):,}  (need ≥ {LOOKBACK})")

# 3️⃣ (Re)train the model
model = load_or_train(df_tr, retrain=False)

# 4️⃣ Inspect the pipeline
print("\n▶️ Pipeline steps:", model.named_steps)
gb = model.named_steps["gb"]
print("▶️ Final estimator:", gb)
print("▶️ n_features_in_:", gb.n_features_in_)
print(f"▶️ Expected      : {len(FEATURES)} × {LOOKBACK} = {len(FEATURES)*LOOKBACK}\n")

# ── NEW: apply trading-window filter ───────────────────────────────────────────
# Morning session: 09:15–11:30, Afternoon session: 12:30–15:25
df_morning   = df_te.between_time("09:15", "10:45")
df_afternoon = df_te.between_time("14:30", "15:30")
df_windowed  = pd.concat([df_morning, df_afternoon]).sort_index()

print(f"Windowed test rows: {len(df_windowed):,}  (from {len(df_te):,} total)")

# 5️⃣ Run back-test on windowed data only
trades, metrics = backtest(
    df_windowed,
    model=model,
    capital=1000000,
    contract_size=1,
    sl_pct=0.0015,
    tp_pct=0.0020,
    trail_pct=0.0015,
    hold_max=10,
    upper=0.486,
    lower=0.596,
    predict_fn=predict_last,
    debug=True,
)

# 6️⃣ Review results
print("Back-test metrics:", metrics)
print("\nFirst few trades:\n", trades.head())


[broker] loaded from /Users/sreejit/PycharmProjects/zerodha-bot/algo/broker.py
[KiteWrapper] initialized: symbol=RELIANCE on exch=NSE
[history] start: days=200, interval=3minute, symbol=IDEA
[history] range UTC-naive: 2024-12-19 14:47:47.276647 → 2025-07-07 14:47:47.276647
[history] token=3677697
[history] got 8639 bars, cursor→2025-03-28 15:30:00
[history] got 8125 bars, cursor→2025-07-04 15:30:00
[history] got 111 bars, cursor→2025-07-07 14:48:00
[history] complete 16875 bars 2024-12-19 14:48:00 → 2025-07-07 14:45:00
ret1 head: [0.0, -0.0013054830287205776, 0.0013071895424836555, 0.0, 0.0026109660574411553]
ema_8 head: [7.66, 7.657777777777778, 7.658271604938271, 7.658655692729766, 7.663398872123151]
ema_21 head: [7.66, 7.659090909090909, 7.659173553719008, 7.659248685199098, 7.661135168362816]
vwap head: [7.656666666666665, 7.655010602663086, 7.655543214758502, 7.655825150482573, 7.6617080111870175]
vol_spike head: [0, 0, 0, 0, 0]
Training rows: 16,624
Testing  rows: 251  (need ≥ 60

In [2]:
trades

,exit_ts,side,entry_price,exit_price,exit_reason,fees,qty,pnl,equity
entry_ts,,,,,,,,,
2025-07-04 15:09:00,2025-07-04 15:15:00,SELL,7.39,7.375220,TP,404.319668,135317,1595.665592,1.001596e+06
2025-07-04 15:18:00,2025-07-04 15:27:00,SELL,7.35,7.361025,SL,405.903369,136271,-1908.291144,9.996874e+05
2025-07-07 09:15:00,2025-07-07 09:18:00,SELL,7.30,7.310950,SL,405.220069,136943,-1904.745919,9.977826e+05
2025-07-07 09:21:00,2025-07-07 09:27:00,SELL,7.35,7.361025,SL,404.537216,135752,-1901.203016,9.958814e+05
2025-07-07 09:30:00,2025-07-07 09:39:00,SELL,7.35,7.335300,TP,402.851278,135494,1588.910522,9.974703e+05
2025-07-07 09:42:00,2025-07-07 09:51:00,BUY,7.32,7.309020,SL,403.561811,136266,-1899.762491,9.955706e+05
2025-07-07 09:54:00,2025-07-07 10:00:00,SELL,7.30,7.310950,SL,403.745563,136379,-1897.095613,9.936735e+05
2025-07-07 10:03:00,2025-07-07 10:24:00,SELL,7.30,7.310950,SL,403.065826,136119,-1893.568876,9.917799e+05
2025-07-07 10:27:00,2025-07-07 10:30:00,SELL,7.31,7.320965,SL,402.388324,135674,-1890.053734,9.898899e+05


In [1]:
# ── Full Reload, Retrain & Back-test (fixed) ────────────────────────
import importlib, pandas as pd
import algo.features as feat_mod
import algo.model    as model_mod
importlib.reload(feat_mod); importlib.reload(model_mod)

from algo.features   import add_indicators, FEATURES
from algo.model      import load_or_train, predict_last, LOOKBACK
from algo.broker     import KiteWrapper
from algo.config     import load_config
from algo.backtester import backtest

# 1️⃣ Pull 200 days of 3-min history and build features
cfg    = load_config()
broker = KiteWrapper(cfg)
hist_all = broker.history(days=200, interval="3minute",
                          tradingsymbol="IDEA")
df_all   = add_indicators(hist_all, debug=True).ffill()

# 2️⃣ Split by *calendar days* instead of row count
n_test_days = 5                          # last 20 trading days ≈ 1500 bars
cutoff      = df_all.index[-1] - pd.Timedelta(days=n_test_days)

df_tr = df_all[df_all.index <  cutoff]    # ~180 days
df_te = df_all[df_all.index >= cutoff]    #  last 20 days  (≳ LOOKBACK rows)

print(f"Training rows: {len(df_tr):,}")
print(f"Testing  rows: {len(df_te):,}  (need ≥ {LOOKBACK})")

# 3️⃣ (Re)train the model
model = load_or_train(df_tr, retrain=False)

# 4️⃣ Inspect the pipeline
print("\n▶️ Pipeline steps:", model.named_steps)
gb = model.named_steps["gb"]
print("▶️ Final estimator:", gb)
print("▶️ n_features_in_:", gb.n_features_in_)
print(f"▶️ Expected      : {len(FEATURES)} × {LOOKBACK} = "
      f"{len(FEATURES)*LOOKBACK}\n")

# 5️⃣ Run back-test
trades, metrics = backtest(
    df_te,
    model=model,
    capital=200_000,
    contract_size=1,
    sl_pct=0.0030,
    tp_pct=0.0065,
    trail_pct=0.0020,
    hold_max=10,
    upper=0.486,
    lower=0.596,
    predict_fn=predict_last,
    debug=True,
)

# 6️⃣ Review results
print("Back-test metrics:", metrics)
print("\nFirst few trades:\n", trades.head())


[broker] loaded from /Users/sreejit/PycharmProjects/zerodha-bot/algo/broker.py
[KiteWrapper] initialized: symbol=RELIANCE on exch=NSE
[history] start: days=200, interval=3minute, symbol=IDEA
[history] range UTC-naive: 2024-12-18 13:11:45.051515 → 2025-07-06 13:11:45.051515
[history] token=3677697
[history] got 8750 bars, cursor→2025-03-28 13:12:00
[history] got 8171 bars, cursor→2025-07-04 15:30:00
[history] empty data for 2025-07-04 15:30:00->2025-07-06 13:11:45.051515, breaking loop
[history] complete 16921 bars 2024-12-18 13:12:00 → 2025-07-04 15:27:00
ret1 head: [0.0, 0.0, 0.001293661060802087, -0.0012919896640826156, 0.0]
ema_8 head: [7.73, 7.73, 7.732222222222223, 7.7317283950617295, 7.731344307270234]
ema_21 head: [7.73, 7.73, 7.730909090909091, 7.730826446280992, 7.7307513148009015]
vwap head: [7.730000000000001, 7.7252230708936835, 7.727830622932173, 7.728579430024545, 7.728689011564924]
vol_spike head: [0, 0, 0, 0, 0]
Training rows: 16,296
Testing  rows: 625  (need ≥ 60)

▶️ 

In [2]:
trades

,exit_ts,side,entry_price,exit_price,exit_reason,fees,qty,pnl,equity
entry_ts,,,,,,,,,
2025-06-30 12:12:00,2025-06-30 12:39:00,SELL,7.47,7.421445,TP,118.362470,26773,1181.600545,201181.600545
2025-06-30 12:42:00,2025-06-30 13:12:00,SELL,7.43,7.420000,TIME,119.081974,27076,151.678026,201333.278571
2025-06-30 13:15:00,2025-06-30 13:45:00,SELL,7.42,7.410000,TIME,119.136244,27133,152.193756,201485.472326
2025-06-30 13:48:00,2025-06-30 14:09:00,SELL,7.41,7.432230,SL,119.445892,27191,-723.901822,200761.570504
2025-06-30 14:12:00,2025-06-30 14:42:00,SELL,7.43,7.450000,TIME,119.167446,27020,-659.567446,200102.003058
...,...,...,...,...,...,...,...,...,...
2025-07-04 13:45:00,2025-07-04 13:57:00,SELL,7.33,7.351990,SL,120.190320,27771,-730.874610,202833.605753
2025-07-04 14:00:00,2025-07-04 14:30:00,SELL,7.35,7.340000,TIME,119.672795,27596,156.287205,202989.892958
2025-07-04 14:33:00,2025-07-04 14:42:00,SELL,7.35,7.372050,SL,119.983613,27617,-728.938463,202260.954495


In [7]:
import sqlite3
import pandas as pd
import datetime as dt

# ─── 1) Open your live trades_old.db ─────────────────────────────────────
db_path = "/algo/trades_old.db"  # adjust to your real path
conn    = sqlite3.connect(db_path, check_same_thread=False)

# ─── 2) Load only 'close' events ────────────────────────────────────
df = pd.read_sql_query(
    "SELECT * FROM trades WHERE event = 'close'",
    conn,
    parse_dates=["timestamp"]
)

# ─── 3) Convert timestamp to IST and filter to today ───────────────
# Make sure timestamp column is timezone-aware (it comes in UTC by default)
df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)
# Convert to IST
df["timestamp_ist"] = df["timestamp"].dt.tz_convert("Asia/Kolkata")
today_ist = dt.datetime.now(dt.timezone(dt.timedelta(hours=5, minutes=30))).date()
df_today = df[df["timestamp_ist"].dt.date == today_ist].copy()

# ─── 4) Compute metrics ─────────────────────────────────────────────
total_trades = len(df_today)
wins         = (df_today["net_pnl"] >  0).sum()
losses       = (df_today["net_pnl"] <= 0).sum()
win_rate     = wins / total_trades * 100 if total_trades else 0
gross_pnl    = df_today["gross_pnl"].sum()
fees         = (df_today["gross_pnl"] - df_today["net_pnl"]).sum()
net_pnl      = df_today["net_pnl"].sum()
final_equity = 200_000 + net_pnl  # adjust if your starting capital differs

metrics = {
    "Total Trades":    total_trades,
    "Winning Trades":  wins,
    "Losing Trades":   losses,
    "Win Rate (%)":    round(win_rate, 2),
    "Gross PnL":       round(gross_pnl, 2),
    "Fees":            round(fees, 2),
    "Net PnL":         round(net_pnl, 2),
    "Equity Final":    round(final_equity, 2),
}

# ─── 5) Pretty‐print them ────────────────────────────────────────────
for k, v in metrics.items():
    print(f"{k:15s}: {v}")


Total Trades   : 5
Winning Trades : 1
Losing Trades  : 4
Win Rate (%)   : 20.0
Gross PnL      : -1106.56
Fees           : 592.09
Net PnL        : -1698.65
Equity Final   : 198301.35


In [10]:
import sqlite3
import pandas as pd
import datetime as dt

# 1) connect to your trades_old.db
db_path = "/algo/trades_old.db"
conn    = sqlite3.connect(db_path, check_same_thread=False)

# 2) read only the close events
df = pd.read_sql_query(
    "SELECT * FROM trades WHERE event = 'close'",
    conn,
    parse_dates=["timestamp"]
)

# 3) parse timestamp as local IST and filter to today
df["timestamp"] = pd.to_datetime(df["timestamp"])    # assume already IST
df["date"]      = df["timestamp"].dt.date

today = dt.datetime.now(dt.timezone(dt.timedelta(hours=5, minutes=30))).date()
df_today = df[df["date"] == today].copy()

# 4) inspect the exact rows you have
print("Rows for today’s closed trades:\n")
print(df_today[[
    "timestamp", "symbol", "qty", "price", "event", "gross_pnl", "net_pnl"
]].to_string(index=False))

# 5) now compute your metrics
total_trades = len(df_today)
wins         = (df_today["net_pnl"] > 0).sum()
losses       = (df_today["net_pnl"] <= 0).sum()
win_rate     = wins / total_trades * 100 if total_trades else 0
gross_pnl    = df_today["gross_pnl"].sum()
fees         = (df_today["gross_pnl"] - df_today["net_pnl"]).sum()
net_pnl      = df_today["net_pnl"].sum()
final_equity = 200_000 + net_pnl  # adjust if your start capital differs

print("\nToday’s live metrics:")
print(f"Total Trades   : {total_trades}")
print(f"Winning Trades : {wins}")
print(f"Losing Trades  : {losses}")
print(f"Win Rate (%)   : {win_rate:.2f}")
print(f"Gross PnL      : {gross_pnl:.2f}")
print(f"Fees           : {fees:.2f}")
print(f"Net PnL        : {net_pnl:.2f}")
print(f"Equity Final   : {final_equity:.2f}")

Rows for today’s closed trades:

                 timestamp symbol   qty    price event  gross_pnl     net_pnl
2025-07-07 10:47:53.004847   IDEA 27210 7.372050 close -599.98050 -718.891479
2025-07-07 12:07:31.473164   IDEA 27212 7.362020 close -599.20824 -718.026917
2025-07-07 12:26:42.445781   IDEA 27061 7.372050 close -596.69505 -715.213345
2025-07-07 13:27:04.974461   IDEA 27025 7.362020 close -595.09050 -713.417016
2025-07-07 15:16:33.276267   IDEA 26958 7.282355 close 1284.41391 1166.902627

Today’s live metrics:
Total Trades   : 5
Winning Trades : 1
Losing Trades  : 4
Win Rate (%)   : 20.00
Gross PnL      : -1106.56
Fees           : 592.09
Net PnL        : -1698.65
Equity Final   : 198301.35
